In [22]:
from pandas import *

data = read_csv('./Train.csv')
# data = DataFrame(train)

data.head()

,ID,customer_id,country_id,tbl_loan_id,lender_id,loan_type,Total_Amount,Total_Amount_to_Repay,disbursement_date,due_date,duration,New_versus_Repeat,Amount_Funded_By_Lender,Lender_portion_Funded,Lender_portion_to_be_repaid,target
0,ID_266671248032267278,266671,Kenya,248032,267278,Type_1,8448.0,8448.0,2022-08-30,2022-09-06,7,Repeat Loan,120.85,0.014305,121.0,0
1,ID_248919228515267278,248919,Kenya,228515,267278,Type_1,25895.0,25979.0,2022-07-30,2022-08-06,7,Repeat Loan,7768.50,0.300000,7794.0,0
2,ID_308486370501251804,308486,Kenya,370501,251804,Type_7,6900.0,7142.0,2024-09-06,2024-09-13,7,Repeat Loan,1380.00,0.200000,1428.0,0
3,ID_266004285009267278,266004,Kenya,285009,267278,Type_1,8958.0,9233.0,2022-10-20,2022-10-27,7,Repeat Loan,2687.40,0.300000,2770.0,0
4,ID_253803305312267278,253803,Kenya,305312,267278,Type_1,4564.0,4728.0,2022-11-28,2022-12-05,7,Repeat Loan,1369.20,0.300000,1418.0,0


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68654 entries, 0 to 68653
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           68654 non-null  object 
 1   customer_id                  68654 non-null  int64  
 2   country_id                   68654 non-null  object 
 3   tbl_loan_id                  68654 non-null  int64  
 4   lender_id                    68654 non-null  int64  
 5   loan_type                    68654 non-null  object 
 6   Total_Amount                 68654 non-null  float64
 7   Total_Amount_to_Repay        68654 non-null  float64
 8   disbursement_date            68654 non-null  object 
 9   due_date                     68654 non-null  object 
 10  duration                     68654 non-null  int64  
 11  New_versus_Repeat            68654 non-null  object 
 12  Amount_Funded_By_Lender      68654 non-null  float64
 13  Lender_portion_F

In [24]:
data.isnull().sum()

ID                             0
customer_id                    0
country_id                     0
tbl_loan_id                    0
lender_id                      0
loan_type                      0
Total_Amount                   0
Total_Amount_to_Repay          0
disbursement_date              0
due_date                       0
duration                       0
New_versus_Repeat              0
Amount_Funded_By_Lender        0
Lender_portion_Funded          0
Lender_portion_to_be_repaid    0
target                         0
dtype: int64

In [25]:
int_colm = data.select_dtypes(include=['float64', 'int64']).columns
data[int_colm] = data[int_colm].fillna(data[int_colm].median())
data = data.drop(['customer_id', 'tbl_loan_id'], axis=1, errors='ignore')

print(data.columns)

Index(['ID', 'country_id', 'lender_id', 'loan_type', 'Total_Amount',
       'Total_Amount_to_Repay', 'disbursement_date', 'due_date', 'duration',
       'New_versus_Repeat', 'Amount_Funded_By_Lender', 'Lender_portion_Funded',
       'Lender_portion_to_be_repaid', 'target'],
      dtype='object')


In [26]:
import numpy as np

data['target'] = np.where(
    data['Lender_portion_to_be_repaid'] > data['Lender_portion_Funded'] * data['Total_Amount'], 1, 0
)

data.head()

,ID,country_id,lender_id,loan_type,Total_Amount,Total_Amount_to_Repay,disbursement_date,due_date,duration,New_versus_Repeat,Amount_Funded_By_Lender,Lender_portion_Funded,Lender_portion_to_be_repaid,target
0,ID_266671248032267278,Kenya,267278,Type_1,8448.0,8448.0,2022-08-30,2022-09-06,7,Repeat Loan,120.85,0.014305,121.0,1
1,ID_248919228515267278,Kenya,267278,Type_1,25895.0,25979.0,2022-07-30,2022-08-06,7,Repeat Loan,7768.50,0.300000,7794.0,1
2,ID_308486370501251804,Kenya,251804,Type_7,6900.0,7142.0,2024-09-06,2024-09-13,7,Repeat Loan,1380.00,0.200000,1428.0,1
3,ID_266004285009267278,Kenya,267278,Type_1,8958.0,9233.0,2022-10-20,2022-10-27,7,Repeat Loan,2687.40,0.300000,2770.0,1
4,ID_253803305312267278,Kenya,267278,Type_1,4564.0,4728.0,2022-11-28,2022-12-05,7,Repeat Loan,1369.20,0.300000,1418.0,1


In [27]:
from sklearn.model_selection import train_test_split

features = ['Total_Amount', 'duration', 'Amount_Funded_By_Lender', 'Lender_portion_Funded']
X = data[features]
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [28]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [29]:
from sklearn.metrics import classification_report, roc_auc_score

y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

report = classification_report(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("Classification Report:\n", report)
print("ROC-AUC Score:", roc_auc)


Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.51      0.59      3973
           1       0.82      0.90      0.86      9758

    accuracy                           0.79     13731
   macro avg       0.75      0.71      0.72     13731
weighted avg       0.78      0.79      0.78     13731

ROC-AUC Score: 0.7349560728811669


In [30]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

model.fit(X_resampled, y_resampled)


LogisticRegression(max_iter=1000)

In [31]:
data = data[['ID', 'target']]
out_file = data.to_csv('./submission2.csv', index = False)
out_file

In [32]:
submission = read_csv('./submission2.csv')
submission

,ID,target
0,ID_266671248032267278,1
1,ID_248919228515267278,1
2,ID_308486370501251804,1
3,ID_266004285009267278,1
4,ID_253803305312267278,1
...,...,...
68649,ID_244559228408267278,1
68650,ID_260062217784267278,1
68651,ID_259137216701267278,1
68652,ID_266801303201267278,0


### Debuggin

In [33]:
# Check if the specific IDs are missing in the dataset
missing_ids = ['ID_269404226088267278', 'ID_247613296713267278', 'ID_271847294122267278', 'ID_308399367770267278', 'ID_25327827841826727']
missing_entries = data[data['ID'].isin(missing_ids)]

# Display missing entries (if any)
print(missing_entries)


Empty DataFrame
Columns: [ID, target]
Index: []
